In [1]:
# https://www.kaggle.com/shivanielakurthy/human-stress-detection-100-accuracy

In [2]:
import pandas as pd
import numpy as np
from numpy import log, dot, e, ndim
from numpy.random import rand
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.metrics import confusion_matrix
#normalizing dataframe
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn import metrics

from tensorflow.keras.utils import to_categorical


from wrapper import t_list


2022-03-11 14:24:45.059390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-11 14:24:45.059412: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
data = pd.DataFrame()
data= pd.read_csv('./SaYoPillow.csv')
data.head()

,sr,rr,t,lm,bo,rem,sr.1,hr,sl
0,93.80,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3
1,91.64,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3
2,60.00,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
3,85.76,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3
4,48.12,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0


In [4]:
data.columns=['snoring_rate', 'respiration_rate', 'body_temperature', 'limb_movement', 'blood_oxygen', \
             'eye_movement', 'sleeping_hours', 'heart_rate', 'stress_level']
data.head()

,snoring_rate,respiration_rate,body_temperature,limb_movement,blood_oxygen,eye_movement,sleeping_hours,heart_rate,stress_level
0,93.80,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3
1,91.64,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3
2,60.00,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
3,85.76,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3
4,48.12,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0


In [5]:
X = data.iloc[:, :8]
y = data['stress_level']

In [6]:
scalerX = MinMaxScaler()
scalerX.fit(X)
X = scalerX.transform(X)
y = y.to_numpy()

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state=8)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((472, 8), (158, 8), (472,), (158,))

In [8]:
num_classes=5
y_train=to_categorical(y_train, num_classes, dtype=np.int8)
y_test=to_categorical(y_test, num_classes, dtype=np.int8)
y_train[:5]

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]], dtype=int8)

In [9]:
X_train[0]

array([0.68654545, 0.48114286, 0.38342857, 0.59466667, 0.42453333,
       0.79822222, 0.04088889, 0.48114286])

In [10]:
# shape None == allow dynamic number of rows; X_train.shape[1] == number of features
X = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), name='inputs')
y = tf.placeholder(dtype=tf.float32, shape=(None, 5), name='targets')

In [11]:
# First hidden layer
# X == features coming in, 30 == number of features == neurons
h1 = tf.layers.dense(X, 8, name='dense1')
h1 = h1 * h1
# Second hidden layer
# h1 == features coming in, 26 == number of neurons == arbitrary
h2 = tf.layers.dense(h1, 6, name='dense2')
h2 = h2 * h2
# Last/output layar always has 1 neuron for binary classification problems
model1 = tf.layers.dense(h2, 5, name='dense3')

model = tf.sigmoid(model1, name='sigmoid')
# model = approx_sigmoid(model)

/tmp/ipykernel_11093/418592932.py:3: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h1 = tf.layers.dense(X, 8, name='dense1')
/home/thuy/.local/lib/python3.8/site-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/tmp/ipykernel_11093/418592932.py:7: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h2 = tf.layers.dense(h1, 6, name='dense2')
/tmp/ipykernel_11093/418592932.py:10: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  model1 = tf.layers.dense(h2, 5, name='dense3')


In [12]:
loss = tf.losses.log_loss(y, model)

# Define training operation
training_op = tf.train.AdamOptimizer(.01).minimize(loss)

In [13]:
# Initialize
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    # Train model with 1000 epochs
    # saver.restore(sess, '/content/drive/MyDrive/Colab Notebooks/result/nn_data/net-1000')
    for epoch in range(1000):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        if (epoch % 100 == 0):
            training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
            test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
            print('epoch:',epoch,' | training loss:', training_loss, ' | test loss:', test_loss)
    saved_path = saver.save(sess, './nn_data/net', global_step=1000)

    # # Use model to classify test data
    # classifications_on_test_data = sess.run(model, feed_dict={X: X_test})
    
    # # Check model accuracy
    # classes = (classifications_on_test_data > .5).astype(int)
    
    # print('\nAccuracy Score:',metrics.accuracy_score(y_test, classes))
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({X: X_test, y: y_test}))
    no_sig_predictions = tf.equal(tf.argmax(model1, 1), tf.argmax(y, 1))
    no_sig_accuracy = tf.reduce_mean(tf.cast(no_sig_predictions, "float"))
    print('Accuracy without sigmoid: ', no_sig_accuracy.eval({X: X_test, y: y_test}))

2022-03-11 14:24:46.560574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-11 14:24:46.560595: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-11 14:24:46.560611: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thuy-ThinkPad-T480): /proc/driver/nvidia/version does not exist
2022-03-11 14:24:46.560761: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


epoch: 0  | training loss: 0.6818228  | test loss: 0.6780427
epoch: 100  | training loss: 7.1913266e-05  | test loss: 7.455492e-05
epoch: 200  | training loss: 3.0347866e-05  | test loss: 3.0256162e-05
epoch: 300  | training loss: 1.8118253e-05  | test loss: 1.813074e-05
epoch: 400  | training loss: 1.1902202e-05  | test loss: 1.1952724e-05
epoch: 500  | training loss: 8.352497e-06  | test loss: 8.415217e-06
epoch: 600  | training loss: 6.1478568e-06  | test loss: 6.2118884e-06
epoch: 700  | training loss: 4.689787e-06  | test loss: 4.751274e-06
epoch: 800  | training loss: 3.6770134e-06  | test loss: 3.7333457e-06
epoch: 900  | training loss: 2.9452426e-06  | test loss: 2.9970438e-06
Accuracy: 1.0
Accuracy without sigmoid:  1.0


# PRE_ENCODE

In [14]:
# Restore session
with tf.Session() as sess:
	saver.restore(sess, './nn_data/net-1000')
	with tf.variable_scope('dense1', reuse=True):
		dense1_kernel_ = sess.run(tf.get_variable('kernel'))
		dense1_bias_ = sess.run(tf.get_variable('bias'))
	with tf.variable_scope('dense2', reuse=True):
		dense2_kernel_ = sess.run(tf.get_variable('kernel'))
		dense2_bias_ = sess.run(tf.get_variable('bias'))
	with tf.variable_scope('dense3', reuse=True):
		dense3_kernel_ = sess.run(tf.get_variable('kernel'))
		dense3_bias_ = sess.run(tf.get_variable('bias'))
	encoded_input_ = X.eval({X: X_test})


INFO:tensorflow:Restoring parameters from ./nn_data/net-1000


In [15]:
print(dense1_kernel_.shape)
print(dense1_bias_.shape)
print(dense2_kernel_.shape)
print(dense2_bias_.shape)
print(dense3_kernel_.shape)
print(dense3_bias_.shape)
print(encoded_input_.shape)

(8, 8)
(8,)
(8, 6)
(6,)
(6, 5)
(5,)
(158, 8)


In [16]:
# Declare the outputs
input_size = encoded_input_.shape[0]
encoded_input = np.empty(shape=(input_size,encoded_input_.shape[1],5), dtype=np.uint64)
dense1_kernel = np.empty(shape=(8,8,5), dtype=np.uint64)
dense1_bias = np.empty(shape=(8,5), dtype=np.uint64)
dense2_kernel = np.empty(shape=(8,6,5), dtype=np.uint64)
dense2_bias = np.empty(shape=(6,5), dtype=np.uint64)
dense3_kernel = np.empty(shape=(6,5,5), dtype=np.uint64)
dense3_bias = np.empty(shape=(5,5), dtype=np.uint64)

In [17]:
# WEIGHTS
print("Weights processing...")
precision = 10
for i in range(8):
	for j in range(8):
		value = round(dense1_kernel_[i, j].item()*precision)
		for t in range(5):
			dense1_kernel[i, j, t] = value % t_list[t]

for i in range(8):
	for j in range(6):
		value = round(dense2_kernel_[i, j].item()*precision)
		for t in range(5):
			dense2_kernel[i, j, t] = value % t_list[t]

for i in range(6):
	for j in range(5):
		value = round(dense3_kernel_[i, j].item()*precision)
		for t in range(5):
			dense3_kernel[i, j, t] = value % t_list[t]

for i in range(8):
	value = round(dense1_bias_[i].item()*(precision**2))
	for t in range(5):
		dense1_bias[i, t] = value % t_list[t]

for i in range(6):
	value = round(dense2_bias_[i].item()*(precision**5))
	for t in range(5):
		dense2_bias[i, t] = value % t_list[t]

for i in range(5):
	value = round(dense3_bias_[i].item()*(precision**11))
	for t in range(5):
		dense3_bias[i, t] = value % t_list[t]

print("Done.")

Weights processing...
Done.


In [18]:
# INPUT
print("Input processing...")
for i in range(input_size):
	for j in range(encoded_input_.shape[1]):
			value = round(encoded_input_[i,j].item()*precision)
			for t in range(5):
				encoded_input[i,j,t] = value % t_list[t]

Input processing...


In [19]:
# Store the encoded tensors
np.save("./output_data/plain_layer_0", encoded_input)
np.save("./nn_data/dense1_kernel", dense1_kernel)
np.save("./nn_data/dense1_bias", dense1_bias)
np.save("./nn_data/dense2_kernel", dense2_kernel)
np.save("./nn_data/dense2_bias", dense2_bias)
np.save("./nn_data/dense3_kernel", dense3_kernel)
np.save("./nn_data/dense3_bias", dense3_bias)

print("Tensors stored")

Tensors stored


# INFERE_PLAIN

In [20]:
import numpy as np
from wrapper import t_list

t_size = len(t_list)

In [21]:
# WEIGHTS
dense1_kernel = np.load("./nn_data/dense1_kernel.npy")
dense1_bias = np.load("./nn_data/dense1_bias.npy")
dense2_kernel = np.load("./nn_data/dense2_kernel.npy")
dense2_bias = np.load("./nn_data/dense2_bias.npy")
dense3_kernel = np.load("./nn_data/dense3_kernel.npy")
dense3_bias = np.load("./nn_data/dense3_bias.npy")

In [22]:
plain_input = np.load("./output_data/plain_layer_0.npy")
examples_count = plain_input.shape[0]
plain_output = np.empty((examples_count,8,5), dtype=np.uint64)

In [23]:
# LAYER 1: dense1
print("Computing layer 1/5...")
temp = np.empty((examples_count,8,t_size), dtype=np.uint64)
for t_index in range(t_size):
	temp[...,t_index] = (plain_input[...,t_index].dot(dense1_kernel[...,t_index])) % t_list[t_index]
	temp[...,t_index] = (temp[...,t_index] + dense1_bias[...,t_index]) % t_list[t_index]
plain_output = temp
temp = None
np.save("./output_data/plain_layer_1", plain_output)

Computing layer 1/5...


In [24]:
# LAYER 2: square activation function
print("Computing layer 2/5...")
plain_output = plain_output*plain_output
for t_index in range(t_size):
	plain_output[...,t_index] = plain_output[...,t_index] % t_list[t_index]
np.save("./output_data/plain_layer_2", plain_output)

Computing layer 2/5...


In [25]:
# LAYER 3: dense2
print("Computing layer 3/5...")
temp = np.empty((examples_count,6,t_size), dtype=np.uint64)
for t_index in range(t_size):
	temp[...,t_index] = (plain_output[...,t_index].dot(dense2_kernel[...,t_index])) % t_list[t_index]
	temp[...,t_index] = (temp[...,t_index] + dense2_bias[...,t_index]) % t_list[t_index]
plain_output = temp
temp = None
np.save("./output_data/plain_layer_3", plain_output)

Computing layer 3/5...


In [26]:
# LAYER 4: square activation function
print("Computing layer 4/5...")
plain_output = plain_output*plain_output
for t_index in range(t_size):
	plain_output[...,t_index] = plain_output[...,t_index] % t_list[t_index]
np.save("./output_data/plain_layer_4", plain_output)

Computing layer 4/5...


In [27]:
# LAYER 5: dense3
print("Computing layer 5/5...")
temp = np.empty((examples_count,5,t_size), dtype=np.uint64)
for t_index in range(t_size):
	temp[...,t_index] = (plain_output[...,t_index].dot(dense3_kernel[...,t_index])) % t_list[t_index]
	temp[...,t_index] = (temp[...,t_index] + dense3_bias[...,t_index]) % t_list[t_index]
plain_output = temp
temp = None
np.save("./output_data/plain_layer_5", plain_output)

Computing layer 5/5...


# INFERE_ENC

In [28]:
import numpy as np
from wrapper import SEAL

In [29]:
SEALobj = SEAL()
q_list = SEALobj.q_list
k_list = SEALobj.k_list
n_parm = SEALobj.n_parm
enc_poly_size = SEALobj.enc_poly_size
q_size = len(q_list)
t_size = len(SEALobj.t_list)

In [30]:
def to_object_dtype(tensoreuint):
	shape = tensoreuint.shape
	tensoreuint.shape = (tensoreuint.size,)
	new = np.empty((tensoreuint.size,), dtype=object)
	for i in range(tensoreuint.size):
		new[i] = int(tensoreuint[i].item())
	new.shape = shape
	return new

In [31]:
# WEIGHTS
dense1_kernel = np.load("./nn_data/dense1_kernel.npy")
dense1_bias = np.load("./nn_data/dense1_bias.npy")
dense2_kernel = np.load("./nn_data/dense2_kernel.npy")
dense2_bias = np.load("./nn_data/dense2_bias.npy")
dense3_kernel = np.load("./nn_data/dense3_kernel.npy")
dense3_bias = np.load("./nn_data/dense3_bias.npy")


In [32]:
# INPUT AND OUTPUT DATA
print("Encrypting the input...")
encrypted_input = np.load("./output_data/plain_layer_0.npy") # not yet encrypted
examples_count = encrypted_input.shape[0]
encrypted_input = SEALobj.encrypt_tensor(encrypted_input) # now it is
np.save("./output_data/enc_layer_0", encrypted_input)
poly_groups_count = encrypted_input.shape[0]//enc_poly_size
encrypted_output = np.empty((encrypted_input.shape[0],8,t_size), dtype=np.uint64)



Encrypting the input...


In [33]:
# LAYER 1: fully connected layer
print("Computing layer 1/5...")
encrypted_output = to_object_dtype(encrypted_input)
dense1_kernel = to_object_dtype(dense1_kernel)
## kernel
temp = np.empty((encrypted_output.shape[0],8,t_size), dtype=object)
for t_index in range(t_size):
	temp[...,t_index] = encrypted_output[...,t_index].dot(dense1_kernel[...,t_index])
encrypted_output = temp
temp = None
## % q
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for size_index in range(2):
				for q_index in range(q_size):
					for n_index in range(n_parm+1):
						axis0 = poly_group_index*enc_poly_size + size_index*q_size*(n_parm+1) + q_index*(n_parm+1) + n_index
						temp = encrypted_output[axis0,axis1,axis2]
						temp = temp % q_list[q_index]
						encrypted_output[axis0,axis1,axis2] = temp
## bias
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for q_index in range(q_size):
				axis0 = poly_group_index*enc_poly_size + ((n_parm+1)*q_index)
				temp = encrypted_output[axis0,axis1,axis2]
				temp = temp + dense1_bias[axis1,axis2].item()*k_list[axis2][q_index]
				temp = temp % q_list[q_index]
				encrypted_output[axis0,axis1,axis2] = temp
np.save("./output_data/enc_layer_1", encrypted_output) # !!!! encrypted_output has dtype=object. To load it use the function at the end of the file !!!!


Computing layer 1/5...


In [34]:

# LAYER 2: square activation function
print("Computing layer 2/5...")
encrypted_output = SEALobj.square_tensor(encrypted_output)
np.save("./output_data/enc_layer_2", encrypted_output)



Computing layer 2/5...


In [35]:
# LAYER 3: fully connected layer
print("Computing layer 3/5...")
encrypted_output = to_object_dtype(encrypted_output)
dense2_kernel = to_object_dtype(dense2_kernel)
## kernel
temp = np.empty((encrypted_output.shape[0],6,t_size), dtype=object)
for t_index in range(t_size):
	temp[...,t_index] = encrypted_output[...,t_index].dot(dense2_kernel[...,t_index])
encrypted_output = temp
temp = None
## % q
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for size_index in range(2):
				for q_index in range(q_size):
					for n_index in range(n_parm+1):
						axis0 = poly_group_index*enc_poly_size + size_index*q_size*(n_parm+1) + q_index*(n_parm+1) + n_index
						temp = encrypted_output[axis0,axis1,axis2]
						temp = temp % q_list[q_index]
						encrypted_output[axis0,axis1,axis2] = temp
## bias
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for q_index in range(q_size):
				axis0 = poly_group_index*enc_poly_size + ((n_parm+1)*q_index)
				temp = encrypted_output[axis0,axis1,axis2]
				temp = temp + dense2_bias[axis1,axis2].item()*k_list[axis2][q_index]
				temp = temp % q_list[q_index]
				encrypted_output[axis0,axis1,axis2] = temp
np.save("./output_data/enc_layer_3", encrypted_output) # !!!! encrypted_output has dtype=object. To load it use the function at the end of the file !!!!


Computing layer 3/5...


In [36]:
# LAYER 4: square activation function
print("Computing layer 4/5...")
encrypted_output = SEALobj.square_tensor(encrypted_output)
np.save("./output_data/enc_layer_4", encrypted_output)


Computing layer 4/5...


In [37]:
# LAYER 5: fully connected layer
print("Computing layer 5/5...")
encrypted_output = to_object_dtype(encrypted_output)
dense3_kernel = to_object_dtype(dense3_kernel)
## kernel
temp = np.empty((encrypted_output.shape[0],5,t_size), dtype=object)
for t_index in range(t_size):
	temp[...,t_index] = encrypted_output[...,t_index].dot(dense3_kernel[...,t_index])
encrypted_output = temp
temp = None
## % q
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for size_index in range(2):
				for q_index in range(q_size):
					for n_index in range(n_parm+1):
						axis0 = poly_group_index*enc_poly_size + size_index*q_size*(n_parm+1) + q_index*(n_parm+1) + n_index
						temp = encrypted_output[axis0,axis1,axis2]
						temp = temp % q_list[q_index]
						encrypted_output[axis0,axis1,axis2] = temp
## bias
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for q_index in range(q_size):
				axis0 = poly_group_index*enc_poly_size + ((n_parm+1)*q_index)
				temp = encrypted_output[axis0,axis1,axis2]
				temp = temp + dense3_bias[axis1,axis2].item()*k_list[axis2][q_index]
				temp = temp % q_list[q_index]
				encrypted_output[axis0,axis1,axis2] = temp
np.save("./output_data/enc_layer_5", encrypted_output) # !!!! encrypted_output has dtype=object. To load it use the function at the end of the file !!!!


Computing layer 5/5...


In [38]:
# DECRYPT
print("Decrypting the output...")
decrypted_output = SEALobj.decrypt_tensor(encrypted_output, examples_count)
encrypted_output = None
np.save("./output_data/decrypted_layer_5", decrypted_output)



Decrypting the output...


# POST_DECODE

In [39]:
import os.path

In [40]:
plain_output_exists = os.path.isfile("./output_data/plain_layer_5.npy")
decrypted_output_exists = os.path.isfile("./output_data/decrypted_layer_5.npy")

In [41]:
def extended_Euclidean_algorithm(a, b):
	b0 = b
	x0, x1 = 0, 1
	if b == 1: return 1
	while a > 1:
		q = a // b
		a, b = b, a%b
		x0, x1 = x1 - q * x0, x0
	if x1 < 0: x1 += b0
	return x1


In [42]:
def chinese_remainder_theorem(array):
	result = 0
	for t_index in range(len(array)):
		result += array[t_index].item() * bezout_coefficients[t_index] * t_product_over_t[t_index]
	return result % t_product

In [43]:
def crt_inverse(tensor):
	examples_count = tensor.shape[0]
	temp = np.empty(tensor.shape[:-1], dtype=object)
	for i in range(examples_count):
		for j in range(5):
			temp[i, j] = chinese_remainder_theorem(plain_output[i, j, :])
			if (temp[i, j]>negative_threshold):
				temp[i, j] = temp[i, j] - t_product
	return temp


In [44]:
# CRT PARAMETERS
# compute the producte of all t, and the threshold for negative numbers:
#   t_product
#   negative_threshold
t_product = 1
for t_index in range(len(t_list)):
	t_product = t_product * t_list[t_index]
negative_threshold = t_product // 2
# compute t_product // t and the Bezout coefficients, for all t: 
#   t_product_over_t
#   bezout_coefficients
t_product_over_t = []
bezout_coefficients = []
for t_index in range(len(t_list)):
	t_product_over_t.append(t_product // t_list[t_index])
	temp = extended_Euclidean_algorithm(t_product_over_t[t_index], t_list[t_index])
	bezout_coefficients.append(temp)



In [45]:
# COMPUTE PREDICTIONS
string = ""
if (plain_output_exists):
	plain_output = np.load("./output_data/plain_layer_5.npy")
if (decrypted_output_exists):
	decrypted_output = np.load("./output_data/decrypted_layer_5.npy")
	if (plain_output_exists):
		if (np.array_equal(plain_output, decrypted_output)):
			print("---- Plain and decrypted outputs coincide ----")
			cn_predictions = crt_inverse(decrypted_output)
			string = "Accuracy with SEAL encryption:"
		else:
			print("---- Plain and decrypted outputs are different. Computing accuracy with plain output ----")
			cn_predictions = crt_inverse(plain_output)
			string = "Accuracy with integer numbers (no encryption):"
	else:
		print("---- plain_layer_5.npy file is missing. Can't compare decrypted and plain outputs ----")
		cn_predictions = crt_inverse(decrypted_output)
		string = "Accuracy with SEAL encryption:"
else:
	print("---- decrypted_layer_5.npy file is missing. Can't compare decrypted and plain outputs ----")
	print("Computing accuracy with plain output...")
	cn_predictions = crt_inverse(plain_output)
	string = "Accuracy with integer numbers (no encryption):"
cn_predictions = np.argmax(cn_predictions, axis=1)


---- Plain and decrypted outputs coincide ----


In [46]:
saver = tf.train.Saver()

# PRINT ACCURACIES
with tf.Session() as sess:
	saver.restore(sess, './nn_data/net-1000')

	tf_guessed_predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
	tf_accuracy = tf.reduce_mean(tf.cast(tf_guessed_predictions, "float"))
	cn_guessed_predictions = tf.equal(cn_predictions, tf.argmax(y, 1))
	cn_accuracy = tf.reduce_mean(tf.cast(cn_guessed_predictions, "float"))


	tf_guessed_predictions_ = tf_guessed_predictions.eval({X: X_test, y: y_test})
	cn_guessed_predictions_ = cn_guessed_predictions.eval({y: y_test})
	swapped_predictions_count = 0
	for i in range(tf_guessed_predictions_.size):
		if (tf_guessed_predictions_[i]!=cn_guessed_predictions_[i]):
			swapped_predictions_count = swapped_predictions_count + 1


	print("Accuracy with tensorflow and no CRT:", tf_accuracy.eval({X: X_test, y: y_test}))
	print(string, cn_accuracy.eval({X: X_test, y: y_test}))
	print("Number of swapped predictions: ", swapped_predictions_count)

INFO:tensorflow:Restoring parameters from ./nn_data/net-1000
Accuracy with tensorflow and no CRT: 1.0
Accuracy with SEAL encryption: 1.0
Number of swapped predictions:  0
